In [22]:
import pandas as pd
import numpy as np
import ast
from tabulate import tabulate
import json
import yaml
import matplotlib.pyplot as plt
import itertools
import dataframe_image as dfi


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [23]:
filename = "REALRUN"  ## insert the name of the file used in the runner.py
df = pd.read_csv(rf"../output/{filename}.csv")

In [24]:
# List of metric names
metric_names = [
    "lift",
    "f1",
    "mse",
    "accuracy",
    "recall",
    "precision",
    "roc_auc",
    "area_under_pr",
    "r2_score",
    "rmse",
]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset["output_metrics"] = dataset["output_metrics"].apply(ast.literal_eval)
dataset["run_time"] = dataset["run_time"].apply(lambda x: np.round(x / 60, 2))
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset["output_metrics"].apply(
        lambda x: x.get(metric) if isinstance(x, dict) else np.nan
    )
    dataset[f"{metric}_std"] = dataset[metric].apply(
        lambda x: np.std(x) if isinstance(x, list) else np.nan
    )
    dataset[metric] = dataset[metric].apply(
        lambda x: np.average(x) if isinstance(x, list) else np.nan
    )

base_cols = [
    "dataset",
    "model",
    "run_time",
    "eval_metric",
    "best_score",
    "score_std",
    "output_metrics",
]

base_cols += [i for i in metric_names]
base_cols += [i + "_std" for i in metric_names]

dataset = dataset[base_cols]

In [25]:
dataset.loc[dataset["model"] == "xgb"].sort_values("dataset").head(10)

,dataset,model,run_time,eval_metric,best_score,score_std,output_metrics,lift,f1,mse,accuracy,recall,precision,roc_auc,area_under_pr,r2_score,rmse,lift_std,f1_std,mse_std,accuracy_std,recall_std,precision_std,roc_auc_std,area_under_pr_std,r2_score_std,rmse_std
2,adult,xgb,110.29,roc_auc,0.928557,0.001319,"{'recall': [0.638623326959847, 0.6619897959183...",3.871984,0.712984,NaN,0.873683,0.651577,0.787306,0.928557,0.829554,NaN,NaN,0.030450,0.007890,NaN,0.003359,0.009964,0.010002,0.001319,0.003736,NaN,NaN
76,adult,xgb,292.69,roc_auc,0.929060,0.001105,"{'recall': [0.655831739961759, 0.6626275510204...",3.880915,0.715100,NaN,0.873560,0.658972,0.781715,0.929060,0.830579,NaN,NaN,0.033750,0.008918,NaN,0.003899,0.009106,0.009904,0.001105,0.004079,NaN,NaN
0,ageconditions,xgb,12.44,f1,0.758796,0.059911,"{'recall': [0.7272727272727273, 0.636363636363...",5.277273,0.758796,NaN,0.926077,0.674892,0.870961,0.953714,0.867463,NaN,NaN,0.238320,0.059911,NaN,0.017031,0.070103,0.061639,0.029420,0.057470,NaN,NaN
79,ageconditions,xgb,52.43,f1,0.791265,0.057157,"{'recall': [0.7272727272727273, 0.818181818181...",5.375685,0.791265,NaN,0.935690,0.711688,0.898778,0.962521,0.886764,NaN,NaN,0.339141,0.057157,NaN,0.014414,0.079380,0.060703,0.030447,0.082745,NaN,NaN
78,breastcancer,xgb,29.15,f1,0.969467,0.015411,"{'recall': [0.9859154929577465, 0.971830985915...",1.565109,0.969467,NaN,0.961341,0.974844,0.965255,0.989702,0.987073,NaN,NaN,0.063435,0.015411,NaN,0.019682,0.023868,0.031057,0.009547,0.017809,NaN,NaN
115,breastcancer,xgb,0.93,f1,0.969421,0.009203,"{'recall': [0.971830985915493, 0.9718309859154...",1.593897,0.969421,NaN,0.961341,0.974804,0.964445,0.991766,0.994674,NaN,NaN,0.010042,0.009203,NaN,0.011881,0.010395,0.019925,0.004636,0.003104,NaN,NaN
5,breastcancer,xgb,13.14,f1,0.970797,0.009948,"{'recall': [0.9577464788732394, 0.971830985915...",1.593897,0.970797,NaN,0.963096,0.974765,0.967342,0.990883,0.993944,NaN,NaN,0.010042,0.009948,NaN,0.012884,0.010562,0.023510,0.006623,0.004587,NaN,NaN
12,creditcard,xgb,24.30,lift,9.654741,0.219316,"{'recall': [0.7575757575757576, 0.818181818181...",9.654741,0.858368,NaN,0.999551,0.788621,0.941987,0.982805,0.854808,NaN,NaN,0.219316,0.021085,NaN,0.000066,0.021561,0.026226,0.008226,0.024581,NaN,NaN
85,creditcard,xgb,275.58,lift,9.674944,0.226646,"{'recall': [0.7373737373737373, 0.808080808080...",9.674944,0.858292,NaN,0.999551,0.788683,0.941861,0.985531,0.852267,NaN,NaN,0.226646,0.022644,NaN,0.000069,0.026930,0.023543,0.006983,0.025416,NaN,NaN
11,diabetes,xgb,61.44,lift,2.495782,0.086792,"{'recall': [0.025088028169014086, 0.0193747247...",2.495782,0.032579,NaN,0.888823,0.016817,0.562761,0.681723,0.234132,NaN,NaN,0.086792,0.009514,NaN,0.000170,0.005059,0.014121,0.005527,0.008144,NaN,NaN


In [26]:
tomax = {
    "mse": False,
    "rmse": False,
    "accuracy": True,
    "recall": True,
    "precision": True,
    "roc_auc": True,
    "area_under_pr": True,
    "lift": True,
    "f1": True,
    "r2_score": True,
}


# Define a function to select the first row based on whether to maximize or minimize the "best_score"
def select_first_row(group):
    metric = group["eval_metric"].iloc[0]
    ascending = not tomax.get(
        metric, True
    )  # If metric not in tomax, assume True (maximize)
    return group.sort_values(by="best_score", ascending=ascending).iloc[0]


# Apply the function to each group
filtered_df = dataset.groupby(["dataset", "model", "eval_metric"]).apply(
    select_first_row
)
# Reset the index to get a new DataFrame
filtered_df = filtered_df.reset_index(drop=True)
# Find the rows that maximize the specified metric for each dataset


# Sort the DataFrame based on whether the metric is to be maximized or not
filtered_df["ascending"] = filtered_df["eval_metric"].map(
    {k: not v for k, v in tomax.items()}
)  # Create a new column for ascending order
dfmax = filtered_df.loc[filtered_df["ascending"] == False].sort_values(
    by=["dataset", "eval_metric", "best_score"], ascending=[False, False, False]
)
dfmin = filtered_df.loc[filtered_df["ascending"] == True].sort_values(
    by=["dataset", "eval_metric", "best_score"], ascending=[False, False, True]
)

best_df = pd.concat([dfmax, dfmin])
best_df.drop(columns=["ascending"], inplace=True)

best_df.head(1)

,dataset,model,run_time,eval_metric,best_score,score_std,output_metrics,lift,f1,mse,accuracy,recall,precision,roc_auc,area_under_pr,r2_score,rmse,lift_std,f1_std,mse_std,accuracy_std,recall_std,precision_std,roc_auc_std,area_under_pr_std,r2_score_std,rmse_std
99,titanic,xgb,63.1,roc_auc,0.886533,0.02697,"{'recall': [0.7101449275362319, 0.779411764705...",2.544672,0.765375,NaN,0.83163,0.716411,0.823094,0.886533,0.858,NaN,NaN,0.134535,0.037007,NaN,0.025735,0.042928,0.044904,0.02697,0.031642,NaN,NaN


# Check State of Run

In [27]:
with open("../configuration/experiment_config.yml", "r") as f:
    config = yaml.safe_load(f)

# Extract the necessary information from the configuration file
included_models = [i.lower() for i in config["include_models"]]
included_datasets = [i.lower() for i in config["include_datasets"]]

# Get all combinations of items from the two lists
combinations_possible = list(itertools.product(included_datasets, included_models))


# Get unique combinations based on 'Column1' and 'Column2'
existing_combinations = df[["dataset", "model"]].drop_duplicates()
# Convert the DataFrame to a list of tuples
existing_combinations = [tuple(row) for row in existing_combinations.values]

missing_combos = [i for i in combinations_possible if i not in existing_combinations]
for i in missing_combos:
    print(f"Missing Combination {i}")

Missing Combination ('titanic', 'resnet')
Missing Combination ('ageconditions', 'resnet')
Missing Combination ('diabetes', 'catboost')
Missing Combination ('diabetes', 'gate')
Missing Combination ('diabetes', 'fttransformer')
Missing Combination ('diabetes', 'tabtransformer')
Missing Combination ('housing', 'gate')
Missing Combination ('housing', 'resnet')
Missing Combination ('housing', 's1dcnn')
Missing Combination ('housing', 'fttransformer')
Missing Combination ('housing', 'tabtransformer')
Missing Combination ('housing', 'gandalf')
Missing Combination ('creditcard', 'catboost')
Missing Combination ('creditcard', 'gate')
Missing Combination ('creditcard', 'resnet')
Missing Combination ('creditcard', 's1dcnn')
Missing Combination ('creditcard', 'fttransformer')
Missing Combination ('creditcard', 'gandalf')
Missing Combination ('heloc', 'resnet')
Missing Combination ('heloc', 'fttransformer')
Missing Combination ('covertype', 'xgb')
Missing Combination ('covertype', 'catboost')
Missi

In [28]:
best_df["dataset"].value_counts()

dataset
iris             13
breastcancer     13
adult            13
titanic          12
ageconditions    12
heloc            11
diabetes          9
housing           7
creditcard        7
covertype         3
Name: count, dtype: int64

# Execution Time Visualization

In [29]:
best_df.sort_values("run_time", ascending=False).head(20)

,dataset,model,run_time,eval_metric,best_score,score_std,output_metrics,lift,f1,mse,accuracy,recall,precision,roc_auc,area_under_pr,r2_score,rmse,lift_std,f1_std,mse_std,accuracy_std,recall_std,precision_std,roc_auc_std,area_under_pr_std,r2_score_std,rmse_std
39,covertype,gandalf,1527.41,accuracy,0.906014,0.007262,"{'accuracy': [0.9089016600537337, 0.9155901925...",NaN,0.908684,NaN,0.906014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006699,NaN,0.007262,NaN,NaN,NaN,NaN,NaN,NaN
38,covertype,categoryembedding,1263.83,accuracy,0.863411,0.009302,"{'accuracy': [0.8540113517072042, 0.8590441621...",NaN,0.867012,NaN,0.863411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008665,NaN,0.009302,NaN,NaN,NaN,NaN,NaN,NaN
40,covertype,tabtransformer,1252.28,accuracy,0.608176,0.020401,"{'accuracy': [0.5938473035247229, 0.5964378491...",NaN,0.635060,NaN,0.608176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.016126,NaN,0.020401,NaN,NaN,NaN,NaN,NaN,NaN
44,creditcard,node,1137.60,lift,9.573313,0.149941,"{'recall': [0.898989898989899, 0.9090909090909...",9.573313,0.168736,NaN,0.981510,0.900392,0.094106,0.983243,0.728929,NaN,NaN,0.149941,0.054108,NaN,0.010352,0.017625,0.032153,0.005713,0.063689,NaN,NaN
1,adult,catboost,1016.28,roc_auc,0.931144,0.000986,"{'recall': [0.651370299553856, 0.6511479591836...",3.902603,0.717295,NaN,0.875035,0.658335,0.788001,0.931144,0.834869,NaN,NaN,0.027832,0.004230,NaN,0.002203,0.006523,0.009410,0.000986,0.003103,NaN,NaN
41,creditcard,autoint,952.01,lift,9.553110,0.189117,"{'recall': [0.8686868686868687, 0.939393939393...",9.553110,0.110934,NaN,0.972199,0.908555,0.059508,0.985149,0.743908,NaN,NaN,0.189117,0.032113,NaN,0.009220,0.034552,0.018546,0.005591,0.017320,NaN,NaN
5,adult,gate,867.75,roc_auc,0.915701,0.002269,"{'recall': [0.8776290630975143, 0.849489795918...",3.688273,0.687556,NaN,0.809619,0.869403,0.568890,0.915701,0.792935,NaN,NaN,0.020604,0.006378,NaN,0.007520,0.011627,0.012785,0.002269,0.004104,NaN,NaN
53,diabetes,resnet,829.88,lift,2.284426,0.099990,"{'recall': [0.0, 0.0, 0.0, 0.0, 0.0], 'precisi...",2.284426,0.000000,NaN,0.888401,0.000000,0.000000,0.656955,0.210096,NaN,NaN,0.099990,0.000000,NaN,0.000023,0.000000,0.000000,0.009810,0.011493,NaN,NaN
50,diabetes,gandalf,709.54,lift,2.510758,0.100110,"{'recall': [0.5752640845070423, 0.653896961690...",2.510758,0.274025,NaN,0.633404,0.619530,0.176183,0.681522,0.232125,NaN,NaN,0.100110,0.002889,NaN,0.025012,0.035605,0.004681,0.005417,0.010594,NaN,NaN
93,titanic,gate,542.06,roc_auc,0.868173,0.027653,"{'recall': [0.8115942028985508, 0.764705882352...",2.482910,0.755746,NaN,0.811431,0.762873,0.749306,0.868173,0.839969,NaN,NaN,0.068732,0.031849,NaN,0.020119,0.046156,0.018810,0.027653,0.030412,NaN,NaN


In [30]:
# Group by the "dataset" column and aggregate "run_time" using the sum function
hyperopt_evals = 1
num_parallel = 2
efficiency_estimate = 0.8
aggregated_df = best_df.groupby("model")["run_time"].sum().reset_index()
# Rename the aggregated column for clarity
aggregated_df = aggregated_df.rename(columns={"run_time": "sum_run_time"})
aggregated_df["total_search_hours_estimate"] = (
    aggregated_df["sum_run_time"] * hyperopt_evals / 60 / num_parallel * 0.8
)
aggregated_df.sort_values("total_search_hours_estimate", ascending=False).head(20)

,model,sum_run_time,total_search_hours_estimate
5,gate,3058.66,20.391067
4,gandalf,2680.87,17.872467
2,categoryembedding,2476.60,16.510667
11,tabtransformer,2424.12,16.160800
8,resnet,2182.86,14.552400
10,tabnet,1933.51,12.890067
7,node,1813.11,12.087400
0,autoint,1790.08,11.933867
9,s1dcnn,1635.60,10.904000
6,mlp,1340.24,8.934933


In [31]:
# Convert DataFrame to ASCII table
class Format:
    end = "\033[0m"
    underline = "\033[4m"


def make_results_table(
    df, dataset_name, display_cols, metric_cols, image_name="", dpi=1000, image_path="/home/boom/sdev/WTabRun/notebooks/tables/",
    image_folder = ""
):
    result_df = df.loc[df["dataset"] == dataset_name].reset_index(drop=True).copy()

    # Create a dictionary to store the indices of the rows with the highest values for each metric column
    max_indices = {}
    for metric in metric_cols:
        max_indices[metric] = result_df[metric].idxmax()

    # Modify all columns with std to include relative std
    result_df["best_score"] = result_df.apply(
        lambda row: f"{row['best_score']:.2f} ± ({row['score_std']:.2f})", axis=1
    )
    for metric in metric_cols:
        result_df[metric] = result_df.apply(
            lambda row: f"{row[metric]:.2f} ± ({row[metric+'_std']:.2f})", axis=1
        )
        # Drop the corresponding std column
        result_df.drop(columns=[metric + "_std"], inplace=True)

    result_df[display_cols].to_csv(rf"{image_path}/{image_folder}/{image_name}.csv")

    return result_df[display_cols]


# Define a custom styling function
def highlight_max_row(s):
    is_max = s == s.max()
    return ["background-color: green" if v else "" for v in is_max]


def highlight_min_row(s):
    is_max = s == s.min()
    return ["background-color: green" if v else "" for v in is_max]

In [32]:
folder = "hyperopt"

# Titanic


In [33]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "titanic"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png" , dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,xgb,0.83 ± (0.03),0.89 ± (0.03),2.54 ± (0.13),0.77 ± (0.04),0.72 ± (0.04),0.82 ± (0.04),0.86 ± (0.03)
1,catboost,0.82 ± (0.03),0.88 ± (0.02),2.48 ± (0.07),0.75 ± (0.05),0.70 ± (0.05),0.80 ± (0.04),0.85 ± (0.02)
2,node,0.82 ± (0.03),0.88 ± (0.02),2.54 ± (0.08),0.77 ± (0.03),0.78 ± (0.05),0.76 ± (0.05),0.86 ± (0.02)
3,fttransformer,0.82 ± (0.02),0.88 ± (0.02),2.54 ± (0.08),0.76 ± (0.03),0.76 ± (0.05),0.76 ± (0.03),0.86 ± (0.02)
4,categoryembedding,0.81 ± (0.02),0.88 ± (0.03),2.57 ± (0.07),0.76 ± (0.03),0.77 ± (0.04),0.75 ± (0.03),0.86 ± (0.02)
5,gandalf,0.81 ± (0.03),0.88 ± (0.02),2.45 ± (0.11),0.75 ± (0.04),0.77 ± (0.05),0.74 ± (0.03),0.85 ± (0.02)
6,tabnet,0.79 ± (0.02),0.87 ± (0.02),2.54 ± (0.09),0.73 ± (0.02),0.73 ± (0.08),0.74 ± (0.07),0.84 ± (0.01)
7,gate,0.81 ± (0.02),0.87 ± (0.03),2.48 ± (0.07),0.76 ± (0.03),0.76 ± (0.05),0.75 ± (0.02),0.84 ± (0.03)
8,autoint,0.80 ± (0.01),0.87 ± (0.02),2.54 ± (0.08),0.75 ± (0.01),0.78 ± (0.02),0.72 ± (0.03),0.84 ± (0.02)
9,s1dcnn,0.81 ± (0.03),0.86 ± (0.02),2.48 ± (0.07),0.73 ± (0.04),0.67 ± (0.05),0.81 ± (0.06),0.83 ± (0.03)


# Housing

In [34]:
display_cols = ["model", "mse", "r2_score", "rmse"]
metric_cols = ["mse", "r2_score", "rmse"]
dataset_name = "housing"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=["r2_score"], axis=0).apply(
    highlight_min_row, subset=["mse", "rmse"], axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,mse,r2_score,rmse
0,catboost,0.18 ± (0.01),0.86 ± (0.01),0.42 ± (0.01)
1,xgb,0.19 ± (0.01),0.86 ± (0.01),0.43 ± (0.01)
2,mlp,0.25 ± (0.01),0.81 ± (0.01),0.50 ± (0.01)
3,node,0.31 ± (0.02),0.77 ± (0.01),0.56 ± (0.02)
4,autoint,0.45 ± (0.01),0.72 ± (0.01),0.67 ± (0.01)
5,tabnet,0.40 ± (0.02),0.70 ± (0.02),0.63 ± (0.02)
6,categoryembedding,0.52 ± (0.02),0.68 ± (0.01),0.72 ± (0.01)


# Heloc

In [35]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "heloc"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,catboost,0.72 ± (0.01),0.80 ± (0.01),1.74 ± (0.02),0.74 ± (0.00),0.77 ± (0.01),0.72 ± (0.01),0.80 ± (0.01)
1,node,0.72 ± (0.00),0.80 ± (0.00),1.73 ± (0.03),0.72 ± (0.01),0.70 ± (0.02),0.74 ± (0.00),0.80 ± (0.01)
2,xgb,0.73 ± (0.01),0.80 ± (0.01),1.73 ± (0.04),0.75 ± (0.00),0.78 ± (0.01),0.72 ± (0.01),0.79 ± (0.01)
3,autoint,0.72 ± (0.01),0.80 ± (0.00),1.72 ± (0.03),0.74 ± (0.01),0.74 ± (0.01),0.73 ± (0.01),0.79 ± (0.01)
4,gandalf,0.73 ± (0.01),0.79 ± (0.01),1.72 ± (0.03),0.74 ± (0.00),0.76 ± (0.01),0.73 ± (0.01),0.79 ± (0.01)
5,s1dcnn,0.69 ± (0.01),0.79 ± (0.01),1.71 ± (0.02),0.66 ± (0.02),0.57 ± (0.03),0.78 ± (0.01),0.79 ± (0.00)
6,categoryembedding,0.71 ± (0.01),0.79 ± (0.00),1.71 ± (0.02),0.72 ± (0.01),0.70 ± (0.04),0.74 ± (0.01),0.79 ± (0.01)
7,tabnet,0.72 ± (0.01),0.79 ± (0.00),1.70 ± (0.02),0.74 ± (0.01),0.74 ± (0.04),0.73 ± (0.01),0.79 ± (0.00)
8,mlp,0.72 ± (0.01),0.79 ± (0.01),1.70 ± (0.02),0.74 ± (0.01),0.77 ± (0.03),0.72 ± (0.01),0.79 ± (0.01)
9,gate,0.72 ± (0.01),0.79 ± (0.01),1.70 ± (0.02),0.73 ± (0.00),0.74 ± (0.01),0.73 ± (0.01),0.79 ± (0.01)


# Diabetes

In [36]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "diabetes"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,gandalf,0.63 ± (0.03),0.68 ± (0.01),2.51 ± (0.10),0.27 ± (0.00),0.62 ± (0.04),0.18 ± (0.00),0.23 ± (0.01)
1,xgb,0.89 ± (0.00),0.68 ± (0.01),2.50 ± (0.09),0.03 ± (0.01),0.02 ± (0.01),0.56 ± (0.01),0.23 ± (0.01)
2,categoryembedding,0.64 ± (0.01),0.68 ± (0.01),2.46 ± (0.08),0.27 ± (0.00),0.62 ± (0.02),0.18 ± (0.00),0.23 ± (0.01)
3,s1dcnn,0.81 ± (0.02),0.65 ± (0.01),2.43 ± (0.12),0.28 ± (0.01),0.33 ± (0.04),0.24 ± (0.02),0.20 ± (0.01)
4,node,0.64 ± (0.03),0.68 ± (0.01),2.43 ± (0.07),0.27 ± (0.01),0.61 ± (0.04),0.18 ± (0.01),0.22 ± (0.01)
5,autoint,0.64 ± (0.01),0.68 ± (0.01),2.39 ± (0.10),0.27 ± (0.00),0.61 ± (0.02),0.17 ± (0.00),0.22 ± (0.01)
6,tabnet,0.64 ± (0.03),0.67 ± (0.01),2.39 ± (0.10),0.27 ± (0.00),0.59 ± (0.05),0.17 ± (0.00),0.21 ± (0.01)
7,mlp,0.89 ± (0.00),0.66 ± (0.01),2.36 ± (0.11),0.03 ± (0.02),0.01 ± (0.01),0.41 ± (0.21),0.22 ± (0.01)
8,resnet,0.89 ± (0.00),0.66 ± (0.01),2.28 ± (0.10),0.00 ± (0.00),0.00 ± (0.00),0.00 ± (0.00),0.21 ± (0.01)


# Creditcard

In [37]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "creditcard"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,categoryembedding,0.97 ± (0.01),0.99 ± (0.01),9.70 ± (0.13),0.10 ± (0.03),0.92 ± (0.02),0.05 ± (0.01),0.74 ± (0.04)
1,xgb,1.00 ± (0.00),0.99 ± (0.01),9.67 ± (0.23),0.86 ± (0.02),0.79 ± (0.03),0.94 ± (0.02),0.85 ± (0.03)
2,node,0.98 ± (0.01),0.98 ± (0.01),9.57 ± (0.15),0.17 ± (0.05),0.90 ± (0.02),0.09 ± (0.03),0.73 ± (0.06)
3,autoint,0.97 ± (0.01),0.99 ± (0.01),9.55 ± (0.19),0.11 ± (0.03),0.91 ± (0.03),0.06 ± (0.02),0.74 ± (0.02)
4,mlp,1.00 ± (0.00),0.98 ± (0.01),9.45 ± (0.19),0.82 ± (0.02),0.78 ± (0.02),0.86 ± (0.03),0.80 ± (0.04)
5,tabtransformer,0.75 ± (0.27),0.93 ± (0.06),8.98 ± (0.80),0.07 ± (0.07),0.89 ± (0.03),0.04 ± (0.04),0.62 ± (0.07)
6,tabnet,0.78 ± (0.39),0.94 ± (0.04),8.76 ± (0.75),0.11 ± (0.07),0.86 ± (0.11),0.06 ± (0.04),0.43 ± (0.20)


# Adult

In [38]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = ["accuracy", "roc_auc", "f1", "recall", "precision", "area_under_pr"]
dataset_name = "adult"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,f1,recall,precision,area_under_pr
0,catboost,0.88 ± (0.00),0.93 ± (0.00),0.72 ± (0.00),0.66 ± (0.01),0.79 ± (0.01),0.83 ± (0.00)
1,xgb,0.87 ± (0.00),0.93 ± (0.00),0.72 ± (0.01),0.66 ± (0.01),0.78 ± (0.01),0.83 ± (0.00)
2,fttransformer,0.81 ± (0.00),0.92 ± (0.00),0.69 ± (0.00),0.86 ± (0.01),0.58 ± (0.00),0.80 ± (0.01)
3,autoint,0.81 ± (0.01),0.92 ± (0.00),0.69 ± (0.01),0.87 ± (0.01),0.57 ± (0.01),0.79 ± (0.01)
4,node,0.81 ± (0.00),0.92 ± (0.00),0.69 ± (0.00),0.86 ± (0.01),0.58 ± (0.01),0.79 ± (0.01)
5,gate,0.81 ± (0.01),0.92 ± (0.00),0.69 ± (0.01),0.87 ± (0.01),0.57 ± (0.01),0.79 ± (0.00)
6,gandalf,0.81 ± (0.00),0.92 ± (0.00),0.69 ± (0.01),0.86 ± (0.00),0.57 ± (0.01),0.79 ± (0.01)
7,categoryembedding,0.81 ± (0.01),0.91 ± (0.00),0.69 ± (0.01),0.87 ± (0.01),0.57 ± (0.01),0.79 ± (0.01)
8,tabnet,0.81 ± (0.01),0.91 ± (0.00),0.68 ± (0.01),0.87 ± (0.01),0.56 ± (0.02),0.78 ± (0.01)
9,mlp,0.85 ± (0.00),0.91 ± (0.00),0.67 ± (0.01),0.63 ± (0.03),0.72 ± (0.01),0.77 ± (0.01)


# Iris

In [39]:
display_cols = ["model", "accuracy", "f1"]
metric_cols = [
    "accuracy",
    "f1",
]
dataset_name = "iris"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,f1
0,autoint,0.99 ± (0.02),0.99 ± (0.02)
1,fttransformer,0.98 ± (0.03),0.98 ± (0.03)
2,gandalf,0.98 ± (0.03),0.98 ± (0.03)
3,node,0.98 ± (0.03),0.98 ± (0.03)
4,tabtransformer,0.98 ± (0.04),0.98 ± (0.04)
5,categoryembedding,0.97 ± (0.02),0.97 ± (0.02)
6,resnet,0.97 ± (0.04),0.97 ± (0.04)
7,tabnet,0.97 ± (0.03),0.97 ± (0.03)
8,xgb,0.97 ± (0.02),0.97 ± (0.03)
9,s1dcnn,0.97 ± (0.04),0.97 ± (0.04)


# Covertype

In [40]:
display_cols = ["model", "accuracy", "f1"]
metric_cols = ["accuracy", "f1"]
dataset_name = "covertype"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,f1
0,gandalf,0.91 ± (0.01),0.91 ± (0.01)
1,categoryembedding,0.86 ± (0.01),0.87 ± (0.01)
2,tabtransformer,0.61 ± (0.02),0.64 ± (0.02)


# Breastcancer

In [41]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "breastcancer"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.sort_values("f1", ascending=False).style.apply(
    highlight_max_row, subset=metric_cols, axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,gate,0.99 ± (0.01),1.00 ± (0.01),1.59 ± (0.01),0.99 ± (0.01),1.00 ± (0.01),0.99 ± (0.01),1.00 ± (0.01)
1,node,0.99 ± (0.01),0.99 ± (0.01),1.59 ± (0.01),0.99 ± (0.01),0.99 ± (0.01),0.98 ± (0.02),1.00 ± (0.00)
2,gandalf,0.99 ± (0.01),1.00 ± (0.01),1.59 ± (0.01),0.99 ± (0.01),0.99 ± (0.01),0.98 ± (0.02),1.00 ± (0.00)
3,fttransformer,0.99 ± (0.01),1.00 ± (0.01),1.59 ± (0.01),0.99 ± (0.01),0.99 ± (0.01),0.99 ± (0.01),1.00 ± (0.00)
4,categoryembedding,0.99 ± (0.01),1.00 ± (0.00),1.59 ± (0.01),0.99 ± (0.01),0.99 ± (0.01),0.99 ± (0.01),1.00 ± (0.00)
5,tabtransformer,0.98 ± (0.01),1.00 ± (0.00),1.59 ± (0.01),0.99 ± (0.01),0.99 ± (0.01),0.98 ± (0.01),1.00 ± (0.00)
6,mlp,0.98 ± (0.01),1.00 ± (0.00),1.59 ± (0.01),0.98 ± (0.01),0.99 ± (0.01),0.98 ± (0.01),1.00 ± (0.00)
7,autoint,0.98 ± (0.01),0.99 ± (0.01),1.59 ± (0.01),0.98 ± (0.01),0.99 ± (0.01),0.98 ± (0.01),1.00 ± (0.00)
8,s1dcnn,0.98 ± (0.01),1.00 ± (0.00),1.59 ± (0.01),0.98 ± (0.01),0.99 ± (0.02),0.98 ± (0.02),1.00 ± (0.00)
9,tabnet,0.97 ± (0.01),0.99 ± (0.01),1.59 ± (0.01),0.98 ± (0.01),0.98 ± (0.01),0.98 ± (0.01),0.99 ± (0.01)


# Ageconditions

In [42]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "ageconditions"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.sort_values("f1", ascending=False).style.apply(
    highlight_max_row, subset=metric_cols, axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,node,0.92 ± (0.04),0.96 ± (0.03),5.19 ± (0.41),0.80 ± (0.09),0.89 ± (0.09),0.73 ± (0.12),0.86 ± (0.08)
1,catboost,0.94 ± (0.02),0.96 ± (0.03),5.47 ± (0.17),0.80 ± (0.05),0.72 ± (0.05),0.89 ± (0.05),0.89 ± (0.05)
2,xgb,0.94 ± (0.01),0.96 ± (0.03),5.38 ± (0.34),0.79 ± (0.06),0.71 ± (0.08),0.90 ± (0.06),0.89 ± (0.08)
3,gandalf,0.91 ± (0.03),0.94 ± (0.02),5.09 ± (0.54),0.77 ± (0.06),0.84 ± (0.04),0.71 ± (0.08),0.81 ± (0.07)
5,mlp,0.91 ± (0.02),0.92 ± (0.02),5.09 ± (0.66),0.73 ± (0.08),0.68 ± (0.11),0.80 ± (0.07),0.80 ± (0.08)
4,s1dcnn,0.91 ± (0.03),0.93 ± (0.03),4.61 ± (0.43),0.73 ± (0.07),0.74 ± (0.06),0.72 ± (0.08),0.73 ± (0.10)
6,autoint,0.89 ± (0.03),0.92 ± (0.03),4.62 ± (0.48),0.73 ± (0.07),0.80 ± (0.06),0.67 ± (0.08),0.75 ± (0.06)
8,gate,0.90 ± (0.04),0.93 ± (0.05),4.83 ± (0.45),0.72 ± (0.11),0.77 ± (0.13),0.69 ± (0.12),0.81 ± (0.08)
9,fttransformer,0.89 ± (0.03),0.92 ± (0.06),4.81 ± (0.74),0.72 ± (0.08),0.81 ± (0.11),0.65 ± (0.08),0.79 ± (0.10)
7,categoryembedding,0.89 ± (0.04),0.92 ± (0.04),4.62 ± (0.74),0.72 ± (0.06),0.82 ± (0.06),0.66 ± (0.09),0.74 ± (0.09)
